# Initialize

In [ ]:
# Base
import requests
import pandas as pd
import datetime as dt

# Plot
import plotly.express as px
import plotly.graph_objects as go

# Data
import collections
from river import datasets

# Tools
from river import utils
from river import stats
from river import anomaly
from river import compose
from river import *

# Stream
from streamz import Stream
from streamz.river import RiverTrain, RiverPredict

# Real Thresh
from scipy.stats import norm

# Load batch of data

In [ ]:
df = pd.read_csv("/Users/marekwadinger/Documents/PhD/Teach/2022-2023/batch_data_processing/exams/data/data_BESS_INV.csv", index_col=0)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.time = pd.to_datetime(df.time)

In [ ]:
df = df.set_index('time')

Get outside temperature

In [ ]:
na_dates = df.index[df['Outside Temperature'].isna()]

try:
    url = "https://archive-api.open-meteo.com/v1/archive?hourly=temperature_2m&timezone=Europe%2FBerlin"
    pos = {'longitude': '49.04', 'latitude': '19.72'}
    date_span = {'start_date': na_dates[1].strftime('%Y-%m-%d'), 
                'end_date': na_dates[-1].strftime('%Y-%m-%d')}

    params={**pos, **date_span}

    response = requests.get(url, params=params)
    df_out_temp = pd.DataFrame(response.json()['hourly'])
    df_out_temp.time = pd.to_datetime(df_out_temp.time, utc=True)
    df_out_temp = df_out_temp.set_index('time')['temperature_2m']
    # Scale
    min_temp = -15
    max_temp = 50
    range_temp = max_temp - min_temp

    df_out_temp = (df_out_temp - min_temp) / range_temp
    # Rasample
    df_out_temp = df_out_temp.resample('1t').interpolate()
    # Combine
    df['Outside Temperature'] = df['Outside Temperature'].combine_first(df_out_temp)
except:
    df = df.drop(columns='Outside Temperature', errors='ignore')

# Test TimeRolling

In [ ]:
mean_ = []
std_ = []
rmean = utils.TimeRolling(stats.Mean(), period=dt.timedelta(hours=1,
                                                            minutes=30))
rvar = utils.TimeRolling(stats.Var(), 
                          period=dt.timedelta(hours=1, minutes=30))

col = 'Inverter Temperature'
for t, x in df.iterrows():
    mean_.append(rmean.update(x[col], t=t.tz_localize(None)).get())
    std_.append((rvar.update(x[col], t=t.tz_localize(None)).get()**(1/2)))

s_mean = pd.Series(mean_, index=df.index)
s_std = pd.Series(std_, index=df.index)

In [ ]:
s_env_pos = s_mean + 3 * s_std
s_env_neg = s_mean - 3 * s_std

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=s_env_pos.index.append(s_env_pos.index[::-1]),
    y= pd.concat([s_env_pos, s_env_neg[::-1]]),
    fill='toself',
    fillcolor='rgba(100,0,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name=f'Mov Mean {col}',
))

fig.add_trace(go.Scatter(
    x=s_mean.index, y=s_mean,
    line_color='rgb(100,0,80)',
    name=f'Mov Mean {col}',
))

We need to define predict_one in order to use Pipelines

In [ ]:
import numpy as np

class HalfSpaceTrees(anomaly.HalfSpaceTrees):
  def predict_proba_one(self, x):
    p = anomaly.HalfSpaceTrees.score_one(self, x)
    return {False: 1.0 - p, True: p}

class QuantileFilter(anomaly.QuantileFilter):
  def __init__(self, anomaly_detector, q: float, protect_anomaly_detector=True):
        super().__init__(
            anomaly_detector=anomaly_detector,
            protect_anomaly_detector=protect_anomaly_detector,
            q=q
        )
  def predict_one(self, *args):
    score = self.score_one(*args)
    return score >= (self.quantile.get() or np.inf)
  
class ThresholdFilter(anomaly.ThresholdFilter):
  def __init__(self, anomaly_detector, threshold: float, protect_anomaly_detector=True):
        super().__init__(
            anomaly_detector=anomaly_detector,
            protect_anomaly_detector=protect_anomaly_detector,
            threshold=threshold
        )
  def predict_one(self, *args):
    score = self.score_one(*args)
    return score >= (self.threshold or np.inf)

# Estimated Threshold using 3 sigma rule-based interval

In [ ]:
rmean = utils.TimeRolling(stats.Mean(), 
                          period=dt.timedelta(hours=1, minutes=30))
rstd = utils.TimeRolling(stats.Var(), 
                          period=dt.timedelta(hours=1, minutes=30))
to_discard = [i for i in df.columns if i != 'SOC']

#model = get_stat
#model |= compose.FuncTransformer(get_rdev)
model = compose.Discard(*to_discard)
model |= ThresholdFilter(
        HalfSpaceTrees(seed=42),
        threshold=0.997
    )

anomaly_samples = []
anomaly_env_pos = []
anomaly_env_neg = []
list_env_pos = []
list_env_neg = []
list_mean = []
list_std = []
l_p = []
l_n = []
scores = []

for t, x in df.iterrows():
    anomaly_samples.append(model.predict_one(x))
    
    # Compute moving mean and std
    x_mean = rmean.update(x, t=t.tz_localize(None)).get()
    list_mean.append(x_mean)
    x_std = rstd.update(x, t=t.tz_localize(None)).get()**(1/2)
    list_std.append(x_std)
    # Compute probability boundaries for normal values
    x_env_pos = (x_mean + 3 * x_std)
    x_env_neg = (x_mean - 3 * x_std)
    list_env_pos.append(x_env_pos)
    list_env_neg.append(x_env_neg)
    # Predict, whether the boundaries fall within normal behavior
    a_env_pos = model.predict_one(x_env_pos)
    a_env_neg = model.predict_one(x_env_neg)
    scores.append(model.score_one(x_env_neg))
    
    if not a_env_pos:
        if l_p:
            l_p.append(l_p[-1])
        else:
            l_p.append(x * np.nan)   
    elif a_env_pos:
        if anomaly_env_pos and anomaly_env_pos[-1] == 1:
            if model.predict_one(l_p[-1]):
                l_p.append(l_p[-1])
            else:
                l_p.append(x_env_pos)
        else:
            l_p.append(x_env_pos)
         
    if not a_env_neg:
        if l_n:
            l_n.append(l_n[-1])
        else:
            l_n.append(x * np.nan)   
    elif a_env_neg:
        if anomaly_env_neg and anomaly_env_neg[-1] == 1:
            if model.predict_one(l_n[-1]):
                l_n.append(l_n[-1])
            else:
                l_n.append(x_env_neg)
        else:
            l_n.append(x_env_neg)
            
    anomaly_env_pos.append(a_env_pos)
    anomaly_env_neg.append(a_env_neg)
    
    
    model = model.learn_one(x)
    
    
s_mean = pd.DataFrame(list_mean, index=df.index)
s_env_pos = pd.DataFrame(list_env_pos, index=df.index)
s_env_neg = pd.DataFrame(list_env_neg, index=df.index)
s_n = pd.DataFrame(l_n, index=df.index)

In [ ]:
print(f"Proportion of anomalous samples: "
      f"{sum(anomaly_samples)/len(anomaly_samples)*100:.02f}%\n"
      f"Total number of anomalous events: "
      f"{sum(pd.Series(anomaly_samples).diff().dropna() == 1)}")

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=s_env_pos.index.append(s_env_pos.index[::-1]),
    y= pd.concat([s_env_pos.SOC, s_env_neg.SOC[::-1]]),
    fill='toself',
    fillcolor='rgba(100,0,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name='Mov Mean SOC',
))

fig.add_trace(go.Scatter(
    x=s_mean.index, y=s_mean.SOC,
    line_color='rgb(100,0,80)',
    name='Mov Mean SOC',
))

fig.add_trace(go.Scatter(
    x=df.index, y=df.SOC,
    line_color='rgb(0,140,120)',
    name='SOC',
))

a = pd.Series(anomaly_env_pos, index=df.index).astype(int).diff()
for x0, x1 in zip(a[a == 1].index, a[a == -1].index):
    fig.add_vrect(x0=x0, x1=x1, fillcolor="yellow", opacity=0.25)

a = pd.Series(anomaly_env_neg, index=df.index).astype(int).diff()
for x0, x1 in zip(a[a == 1].index, a[a == -1].index):
    fig.add_vrect(x0=x0, x1=x1, fillcolor="yellow", opacity=0.25)
    
a = pd.Series(anomaly_samples, index=df.index).astype(int).diff()
for x0, x1 in zip(a[a == 1].index, a[a == -1].index):
    fig.add_vrect(x0=x0, x1=x1, fillcolor="red", opacity=0.25)
    
fig.add_trace(go.Scatter(
    x=df.index, y=s_n.SOC,
    line_color='rgb(0,0,100)', 
    name='Lower Threshold 1', fillcolor='rgba(0,0,100, 0.1)', fill="tozeroy"
))

fig.add_trace(go.Scatter(
    x=df.index, y=pd.Series(anomaly_env_neg).astype(int),
    line_color='rgb(160,0,0)',
    name='Neg Anomaly Env',
))

fig.add_trace(go.Scatter(
    x=df.index, y=scores,
    line_color='rgb(100,100,0)',
    name='Score',
))

# Real physical threshold using Gaussian Scorer

In [ ]:
class GaussianScorer(anomaly.GaussianScorer):
    def learn_one(self, x, **kwargs):
        self.gaussian.update(x, **kwargs)
        return self

    def score_one(self, x, t=None):
        if self.gaussian.n_samples < self.grace_period:
            return 0
        return 2 * abs(self.gaussian.cdf(x) - 0.5)

In [ ]:
df.plot(kind='hist', backend='plotly')

In [ ]:
df = df[df.index < '2022-03-27']

In [ ]:
df = pd.read_csv("data/solar_prediction.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df['dev'] = (df['real'] - df['pred']).abs()
df.dev = (df.dev - df.dev.min()) / (df.dev.max() - df.dev.min())

In [ ]:
len(df)

In [ ]:
X_y = datasets.WaterFlow()
df = pd.DataFrame(X_y, columns=['time', 'flow'])
df.time = df.time.apply(lambda x: x['Time'].replace(tzinfo=None))
df.time = pd.to_datetime(df.time)
df = df.set_index('time')
df.flow = (df.flow - df.flow.min()) / (df.flow.max() - df.flow.min())
df.head()

In [ ]:
threshold = 0.99735
#to_discard = [i for i in df.columns if i != 'SOC']
window = dt.timedelta(hours=24*7)
model = GaussianScorer(
                grace_period=10,
                period=window
            )

model_inv = GaussianScorer(
                grace_period=10,
                period=window
            )

col = 'SOC'
anomaly_samples = []
anomaly_samples_ = []
scores = []
scores_ = []
list_thresh_pos = []
list_thresh_neg = []
mus = []
mus_ = []
sigmas = []
sigmas_ = []
samples = []

for t, x in df.iterrows():
    t = t.tz_localize(None)
    x = x[col]
    score = model.score_one(x); scores.append(score)
    samples.append(model.gaussian.n_samples)
    is_anomaly = 1 if score > threshold else 0
    anomaly_samples.append(is_anomaly)    
    score_ = model_inv.score_one(-x); scores_.append(score_)
    #anomaly_samples.append(model_inv.classify(score_))
    is_anomaly_ = 1 if score_ > threshold else 0
    anomaly_samples_.append(is_anomaly_)
    
    kwargs = {'loc': model.gaussian.mu, 
              'scale': model.gaussian.sigma}
    sigmas.append(model.gaussian._var.get())
    mus.append(model.gaussian.mu)
    real_thresh = norm.ppf((threshold/2 + 0.5), **kwargs)
    real_thresh = real_thresh if real_thresh < 1 else 1
    list_thresh_pos.append(real_thresh)
    
    kwargs_inv = {'loc': model_inv.gaussian.mu, 
              'scale': model_inv.gaussian.sigma}
    sigmas_.append(model_inv.gaussian._var.get())
    mus_.append(model_inv.gaussian.mu)
    real_thresh = -norm.ppf((threshold/2 + 0.5), **kwargs_inv)
    real_thresh = real_thresh if real_thresh > 0 else 0
    list_thresh_neg.append(real_thresh)
    # the sample before previous is anomalous
    if not is_anomaly or (is_anomaly and 
                          sum(anomaly_samples[-60:-1]) / 
                          len(anomaly_samples[-60:-1]) > 0.9973):
        model = model.learn_one(x, **{'t': t})
    if not is_anomaly_ or (is_anomaly_ and 
                          sum(anomaly_samples_[-60:-1]) / 
                          len(anomaly_samples_[-60:-1]) > 0.9973):
        model_inv = model_inv.learn_one(-x, **{'t': t})
    
s_thresh_pos = pd.Series(list_thresh_pos, index=df.index)
s_thresh_neg = pd.Series(list_thresh_neg, index=df.index)

In [ ]:
s_mean = pd.Series(mus, index=df.index)
s_std = pd.Series(sigmas, index=df.index)

In [ ]:
s_env_pos = s_mean + 3 * s_std**0.5
s_env_neg = s_mean - 3 * s_std**0.5

In [ ]:
text = (f"Sliding window: {window}<br>"
        f"Proportion of anomalous samples: "
        f"{sum(anomaly_samples)/len(anomaly_samples)*100:.02f}%<br>"
        f"Total number of anomalous events: "
        f"{sum(pd.Series(anomaly_samples).diff().dropna() == 1)}")

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index, y=abs(df[col]),
    line_color='rgb(0,140,120)',
    name=col, showlegend=True
))

  
fig.add_trace(go.Scatter(
    x=df.index, y=([1] if s_thresh_pos.max(skipna=True) < 1 
                   else [s_thresh_pos.max(skipna=True)])*len(df),
    line_color='rgba(100,100,100, 0)', 
    name='Threshold', legendgroup='thresh', showlegend=False
))

fig.add_trace(go.Scatter(
    x=s_thresh_pos.index, y=s_thresh_pos,
    line_color='rgba(100,0,0,0.25)',
    fillcolor='rgba(100,0,0, 0.1)', fill="tonexty",
    name='Threshold', legendgroup='thresh', showlegend=True
))

fig.add_trace(go.Scatter(
    x=s_thresh_neg.index, y=s_thresh_neg,
    line_color='rgba(100,0,0,0.25)', 
    fillcolor='rgba(100,0,0, 0.1)', fill="tozeroy",
    name='Threshold', legendgroup='thresh', showlegend=False
))

a = pd.Series(anomaly_samples, index=df.index).astype(int).diff()
b = a[a == 1].resample('1d').sum()
for x0, x1 in zip(a[a == 1].index, a[a == -1].index):
    fig.add_vrect(x0=x0, x1=x1, line_color="red", fillcolor="red", 
                  opacity=0.25)
'''
#fig.add_annotation(text=text, align='left',
#                  xref="paper", yref="paper",
#                  x=0, y=1.2, showarrow=False)
'''
fig.update_layout(
    yaxis_title="Normalized Temperature",
    yaxis_title_standoff = 0,
    yaxis_range=[0,1],
    xaxis_tickangle=60,
    xaxis_tickfont_size=9,
    xaxis_tickvals=b[b > 0].index,
    
    font_family="Times New Roman",
    font_size=9,
    
    autosize=True,
    height=90*3,
    width=120*3,
    margin=dict(l=40, r=15, t=0, b=0),
    bargap=0,
        
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()

In [ ]:
file_name = (f"{col.replace(' ', '_')}_"
             f"{int(window.total_seconds()/60/60)}_hours_sliding")

In [ ]:
fig.write_html(f"{file_name}_thresh.html")
fig.write_image(f"{file_name}_thresh.pdf")

In [ ]:
fig.add_trace(go.Scatter(
    x=s_env_pos.index.append(s_env_pos.index[::-1]),
    y= pd.concat([s_env_pos, s_env_neg[::-1]]),
    fill='toself',
    fillcolor='rgba(100,0,80,0.2)',
    line_color='rgba(255,255,255,0)',
    showlegend=False,
    name=f'Mov Mean {col}',
))

fig.add_trace(go.Scatter(
    x=s_mean.index, y=s_mean,
    line_color='rgb(100,0,80)',
    name=f'Mov Mean {col}',
))

In [ ]:
fig.write_html(f"{file_name}_mean.html")
fig.write_image(f"{file_name}_mean.pdf")

In [ ]:
a = pd.Series(anomaly_samples, index=df.index).astype(int).diff()
for x0, x1 in zip(a[a == 1].index, a[a == -1].index):
    fig.add_vrect(x0=x0, x1=x1, line_color="red", fillcolor="red", 
                  opacity=0.25)
fig.show()

In [ ]:
fig.write_html(f"{file_name}_anomalies.html")
fig.write_image(f"{file_name}_anomalies.pdf")

Explore value score relationship

In [ ]:
l = []
for i in [*map(lambda x: x/1000, range(200, 450))]:
    x = i
    l.append(model.score_one(x))
px.line(x=[*map(lambda x: x/1000, range(200, 450))],y=l)

Feature Engineering

In [ ]:
def get_input(x):
    return x

def get_rmean(x):
    x_ = x.copy()
    t = x_.name.tz_localize(None)
    if rmean._latest < t:
        rmean.update(x_, t=t)
    return compose.Prefixer('mean_').transform_one(rmean.get())

def get_rstd(x):
    x_ = x.copy()
    t = x_.name.tz_localize(None)
    if rstd._latest < t:
        rstd.update(x_, t=t)
        if any(rstd.get() < 0):
            print(f'Heuston we have a problem! Var is: \n{rstd.get()}')
        
    return compose.Prefixer('std_').transform_one(rstd.get())

def get_stat(x):
    return {**x , **get_rmean(x), **get_rstd(x)}

def get_rdev(x):
    in_ = get_input(x)
    mean_ = get_rmean(x) 
    new_ = {}
    for key in in_:
        if f'mean_{key}' in mean_:
            new_[key] = in_[key] - mean_[f'mean_{key}']

    return {**x, **compose.Prefixer('dev_').transform_one(new_)}

Inverse Tree Score

In [ ]:
self = HalfSpaceTrees(seed=42, n_trees=1, height=2,)
for t, x in df.head(800).iterrows():
    self = self.learn_one({"SOC": x.SOC})

In [ ]:
df.iloc[801].SOC

In [ ]:
score = 0.0
for t in self.trees:
    for depth, node in enumerate(t.walk({"SOC": df.iloc[801].SOC})):
        score += node.r_mass * 2**depth
        print(score)
        if node.r_mass < self.size_limit:
            break

# Normalize the score between 0 and 1
score /= self._max_score
print(score, self._max_score)
score = 1-score
score

In [ ]:
score = 1 - score; score

In [ ]:
score = score * self._max_score; score

In [ ]:
def walk(self, x, until_leaf=True):
        """Iterate over the nodes of the path induced by x."""
        yield self
        try:
            yield from self.next(x).walk(x, until_leaf)
        except KeyError:
            if until_leaf:
                _, node = self.most_common_path()
                yield node
                yield from node.walk(x, until_leaf)

In [ ]:
next(t.walk({"SOC": df.iloc[801].SOC}))

In [ ]:
for t in self.trees:
    for depth, node in enumerate(t.walk({"SOC": df.iloc[801].SOC})):
        score -= node.r_mass * 2**depth
        print(node.r_mass, depth, node.r_mass * 2**depth)
        if node.r_mass < self.size_limit:
                    break
            

In [ ]:
score

In [ ]:
self._max_score

In [ ]:
model.n_trees * model.window_size * (2 ** (model.height + 1) - 1)

In [ ]:
model.trees

Inverse Gaussian

In [ ]:
from scipy.stats import norm

In [ ]:
self = anomaly.GaussianScorer()
self_inv = anomaly.GaussianScorer()
for t, x in df.head(1000).iterrows():
    self = self.learn_one(None, x.SOC)
    self_inv = self_inv.learn_one(None, -x.SOC)

In [ ]:
x = df.iloc[830].SOC; x

In [ ]:
p = self.gaussian.cdf(x); p

In [ ]:
score = self.score_one(None, x); score

In [ ]:
kwargs = {'loc': self.gaussian.mu, 'scale': self.gaussian.sigma}

In [ ]:
norm.ppf((score/2 + 0.5), **kwargs)

In [ ]:
p = self_inv.gaussian.cdf(x); p

In [ ]:
score = self_inv.score_one(None, x); score

In [ ]:
kwargs_inv = {'loc': self_inv.gaussian.mu, 'scale': self_inv.gaussian.sigma}

In [ ]:
norm.ppf((score/2 + 0.5), **kwargs_inv)

We will look for original value for transformed threshold

In [ ]:
norm.ppf((0.997/2 + 0.5), **kwargs)

In [ ]:
-norm.ppf((0.997/2 + 0.5), **kwargs_inv)

In [ ]:
kwargs

In [ ]:
0.997/2 - 0.5

In [ ]:
norm.ppf((0.856), **kwargs)

In [ ]:
df.head(1000).SOC.plot(backend='plotly')